In [1]:
from tqdm import trange

import torch
import torch.nn as nn
%matplotlib inline
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [2]:
batch_size=64

train_dataset = datasets.MNIST(root='./', download=True, train=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./', download=True, train=False, transform=transforms.ToTensor())

x_train = train_dataset.data.float()
y_train = train_dataset.targets
y_train_ohe = torch.nn.functional.one_hot(y_train)

train_dl = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size)
test_dl = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size)

In [3]:
class VAE(nn.Module):
    def __init__(self, hidden=5):
        super().__init__()
        self.hidden = hidden
        self.encoder = nn.Sequential(
            nn.Linear(784, self.hidden**2),
            nn.ReLU(),
            nn.Linear(self.hidden**2,self.hidden*2)
            
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.hidden, self.hidden**2),
            nn.ReLU(),
            nn.Linear(self.hidden**2, 784),
            nn.Sigmoid()
        )
    def param_trick(self, mu, var):
        if self.training:
            std = var.mul(1/2).exp_()
            eps = std.data.new(std.size()).normal_()
            return mu + (eps * std)
        
        else:
            return mu
        
    def forward(self, x):

        x = x.reshape(-1,784)
        
        # Feed through the encoder
        x = self.encoder(x) # 1 x 10
        x = x.reshape(-1, 2, self.hidden)
        
        mu = x[:,0, :]
        var = x[:, 1, :]
        # Sample z ~ mu + e*var
        x = self.param_trick(mu, var)
        return self.decoder(x)
        
#         mu_var = self.param_trick()
        
def loss_function(x_hat, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(x_hat, x.view(-1, 784), reduction='sum')
    KLD = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))

    return BCE + KLD

In [4]:
model = VAE(5)
max_epochs = 10

In [5]:
img = next(iter(train_dl))[0][0].reshape(28,28).to(device)
output = model(img)
output.shape

torch.Size([1, 784])

In [20]:
for image, label in train_dl:
    output = model(image)
    loss = loss_function()
